# Options Trading

This notebook attempts to analyze the various options strategies played during the time period, and their relative success rate.

In [ ]:
import datetime
import re

import robinhood.logic.dataframe
from robinhood.models.option import OptionType


def get_options_trades(from_date: str, to_date: str):
    trades = robinhood.logic.dataframe.trades.get(
        from_date=from_date,
        to_date=to_date,
    )

    regex = re.compile(
        r'(?P<ticker>[A-Z]+)'
        r'(?P<expiration_date>\d{6})'
        r'(?P<type>[PC])'
        r'(?P<strike_price>\d{8})'
    )
    output = trades[
        trades.apply(lambda row: len(row['Name']) > 6, axis=1)
    ]

    # TODO: We don't currently do any advanced multi-leg strategies (or option writing).
    # However, if we want to analyze that, we should use the strategy type, rather than
    # the individual legs.
    output['Type'] = output.apply(
        lambda row: (
            OptionType.CALL
            if regex.match(row['Name']).group('type') == 'C'
            else OptionType.PUT
        ),
        axis=1,
    )

    output['Expiration Date'] = output.apply(
        lambda row: datetime.datetime.strptime(
            regex.match(row['Name']).group('expiration_date'),
            '%y%m%d'
        ).date(),
        axis=1,
    )

    output['Strike Price'] = output.apply(
        lambda row: round(
            int(regex.match(row['Name']).group('strike_price')) / 1000.0,
            2,
        ),
        axis=1,
    )

    # After we finish processing names, we can simplify them.
    output['Name'] = output.apply(
        lambda row: regex.match(row['Name']).group('ticker'),
        axis=1,
    )

    # Lastly, reorder the columns
    columns = output.columns.tolist()
    columns = columns[0:1] + columns[-3:] + columns[1:-3]

    return output[columns]


option_trades = get_options_trades('2020-01-01', '2020-12-31')

## Accuracy

Unlike trading stocks, [trading options is a zero-sum game](https://www.investopedia.com/terms/z/zero-sumgame.asp). As such, you're naturally going to win some and lose some. Since net earnings can often mask the individual trader's success rate at betting on the market, this calculates how often strategies were accurate.

In [ ]:
from collections import defaultdict

import pandas as pd

from robinhood.logic.quotes import get_closing_price_for_stocks


def get_underlying_stock_data(trades: pd.DataFrame) -> pd.DataFrame:
    # Compress data, so we can minimize on network calls
    stocks = defaultdict(list)
    for trade in trades:
        stocks[trade['Name']].append(trade['Expiration Date'])

    quotes = get_closing_price_for_stocks(**stocks)

    # TODO: Would be super cool if we can capture the underlying stock price when you bought the option.
    return pd.concat(
        [
            trades['Name'],
            trades['Type'],
            trades['Expiration Date'],
            trades['Strike Price'],
            trades['Date Bought'],

        ],
        axis=1,
    )


### Strategy

How capable are you for choosing the right options trading strategy?

In [ ]:
def calculate_strategy_accuracy(trades: pd.DataFrame) -> float:
    return trades.apply(
        lambda row: int(row['Earnings'] > 0),
        axis=1,
    ).sum() / len(trades)


print(
    '{percentage}% ({count} / {total})'.format(
        percentage=calculate_strategy_accuracy(option_trades),
        count=int(calculate_strategy_accuracy(option_trades) * len(option_trades)),
        total=len(option_trades),
    )
)

### Direction

How capable are you at guessing which way the stock is going to trend towards?

In [ ]:
from collections import defaultdict

import pandas as pd

from robinhood.logic.quotes import get_closing_price_for_stocks


def calculate_direction_accuracy(trades: pd.DataFrame) -> float:
    # Compress data, so we can minimize on network calls
    stocks = defaultdict(list)
    for trade in trades:
        stocks[trade['Name']].append(trade['Expiration Date'])

    quotes = get_closing_price_for_stocks(**stocks)

    def is_accurate(row):
        row['Name']

    return trades.apply(is_accurate, axis=1)


# calculate_direction_accuracy(option_trades)
option_trades.head()

### Strike Price

TODO